In [ ]:
!pip install transformers -U


In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")
data.head()

<ipython-input-3-dfd017dc2c35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")


,Abstract,Question,Answer,Label,TemporalAspect
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes,Frequency
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes,Frequency
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes,Typical Time
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes,Typical Time
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes,Event Duration


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
import torch
from transformers import AutoTokenizer, DebertaForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-Base")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-Base")


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-Base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')


In [ ]:
import numpy as np  # Import numpy for NaN handling

# Map string labels to numerical values
label_mapping = {"yes": 1, "no": 0}
data["Label"] = data["Label"].map(label_mapping)

data["Abstract"] = data["Abstract"].fillna("")
data["Question"] = data["Question"].fillna("")
data["Answer"] = data["Answer"].fillna("")

# Concatenate the strings for context, question, and answer
X = ["[CLS] " + str(abstract) + " [SEP] " + str(question) + " [SEP] " + str(answer) + " [SEP]" for abstract, question, answer in zip(data["Abstract"], data["Question"], data["Answer"])]

y = list(data["Label"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenizing a single sequence (context, question, answer concatenated)
X_train_tokenized = tokenizer(
    X_train,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)

X_val_tokenized = tokenizer(
    X_val,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",  # Assuming you're using PyTorch
)



In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [ ]:

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
!pip install optuna

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
import torch



def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 3, 10)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    args = TrainingArguments(
        output_dir="output",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate

    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    try:
        trainer.train()
        result = trainer.evaluate()
        return result['eval_loss']
    except RuntimeError as e:
        print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
        torch.cuda.empty_cache()
        return float('inf')

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

best_params = study.best_params


[I 2023-12-19 15:05:14,281] A new study created in memory with name: no-name-c04f57a3-4396-43c3-a5f2-06010293dda8
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.693000


<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-12-19 15:09:34,249] Trial 0 finished with value: 0.6846777200698853 and parameters: {'num_epochs': 3, 'batch_size': 16, 'learning_rate': 0.0005098101577387284}. Best is trial 0 with value: 0.6846777200698853.


<class 'transformers.trainer_utils.EvalPrediction'>


...

<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.689000
1000,0.702000
1500,0.698800
2000,0.701300
2500,0.700300
3000,0.692500


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-8-7c437346b76d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory output/checkpoint-1500 already exists and is non-empty.Saving will p

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-12-19 16:19:00,174] Trial 4 finished with value: 0.6848927140235901 and parameters: {'num_epochs': 3, 'batch_size': 4, 'learning_rate': 0.000324396723213272}. Best is trial 0 with value: 0.6846777200698853.


<class 'transformers.trainer_utils.EvalPrediction'>


In [ ]:
import json

# Save best_params to a JSON file
with open('best_hyperparameters.json', 'w') as file:
    json.dump(best_params, file)

In [ ]:
# Define Trainer
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
np.set_printoptions(suppress=True)


In [ ]:
context = "Wallace, 38, called Gastonia home from the age of 8 until she graduated from Hunter Huss High School in 1983.	"
question = "How often does Brewer talk about the legal system marketplace?	"
answer = "pushkin liked to speak at high noon usually	"
input_sequence = f"[CLS] {context} [SEP] {question} [SEP] {answer} [SEP]"

# text = "go to hell"
tokenized_input = tokenizer(input_sequence,padding = True, truncation = True, return_tensors='pt').to('cuda')
# Make predictions
with torch.no_grad():
    outputs = model(**tokenized_input)

# Get the predicted label
logits = outputs.logits
predicted_label = "yes" if logits.argmax() == 1 else "no"
print("Predicted Label:", predicted_label)

In [ ]:
! pip install accelerate -U
! pip install -U transformers
import os
os._exit(00)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.2 MB/s eta 0:00:00
